In [108]:
import re
import requests
import urllib.parse
from bs4 import BeautifulSoup

This function searches in a web page.

In [109]:
def get_links(url, *args, **kwargs):
    """
    Search in a web page for arbitrary items.
    The url argument is required, everything else is passed to a Beautiful Soup find call.
    """
    referrer = urllib.parse.urlparse(url)

    def canonical(link):
        u = urllib.parse.urlparse(link, scheme=referrer.scheme)
        if u.netloc == '':
            u = u._replace(netloc=referrer.netloc)
        return u.geturl()

    links = BeautifulSoup(
        requests.get(url).text
    ).find_all(
        'a', *args, **kwargs
    )
    
    return [canonical(l['href']) for l in links]

First, find the latest page

In [110]:
VACANCIES_URL = 'https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/jobsandvacanciesintheuk/previousReleases'
latest_page_link = get_links(VACANCIES_URL, string=re.compile(r'Vacancies and jobs in the UK'))[0]
latest_page_link

'https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/jobsandvacanciesintheuk/september2023'

In [111]:
latest_csv = get_links(latest_page_link, href=re.compile(r'&format=csv$'))[1]
latest_csv

'https://www.ons.gov.uk/generator?uri=/employmentandlabourmarket/peopleinwork/employmentandemployeetypes/bulletins/jobsandvacanciesintheuk/september2023/5fe14de6&format=csv'

Save the latest file as a csv

In [112]:
with open('../../data/raw/vacancies-by-sector.csv', 'wb') as f:
    f.write(requests.get(latest_csv).content)